In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
# pd.set_option("max_columns", None)
# pd.set_option('max_colwidth', None)
from collections import defaultdict

nested_dict = lambda: defaultdict(nested_dict)

from pathlib import Path
home = str(Path.home())
base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_1/')
base_path

'/home/lorenzp/workspace/multiLID/data/extract/run_1/'

# from .pt

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')

sklearn.exceptions.ConvergenceWarning('ignore')

In [3]:
def split_data(characteristics, characteristics_adv, noise, test_size=0.2, random_state=42):
    
    shape_adv = np.shape(characteristics_adv)[0]
    shape_char = np.shape(characteristics)[0]
    
    adv_X_train_val, adv_X_test, adv_y_train_val, adv_y_actual = train_test_split(characteristics_adv, np.ones(shape_adv),   test_size=test_size, random_state=random_state)
    b_X_train_val, b_X_test, b_y_train_val, b_y_actual         = train_test_split(characteristics,     np.zeros(shape_char), test_size=test_size, random_state=random_state)
    adv_X_train, adv_X_val, adv_y_train, adv_y_val             = train_test_split(adv_X_train_val,     adv_y_train_val,      test_size=test_size, random_state=random_state)
    b_X_train, b_X_val, b_y_train, b_y_val                     = train_test_split(b_X_train_val,       b_y_train_val,        test_size=test_size, random_state=random_state)

    adv_X_train, adv_X_val, adv_y_train, adv_y_val           = train_test_split(adv_X_train_val,     adv_y_train_val,      test_size=test_size, random_state=random_state)
    b_X_train, b_X_val, b_y_train, b_y_val                   = train_test_split(b_X_train_val,       b_y_train_val,        test_size=test_size, random_state=random_state)


    X_train = np.concatenate(( b_X_train, adv_X_train) )
    y_train = np.concatenate(( b_y_train, adv_y_train) )

    X_test = np.concatenate( (b_X_test, adv_X_test, b_X_val, adv_X_val) )
    y_actual = np.concatenate( (b_y_actual, adv_y_actual, b_y_val, adv_y_val) )


    return X_train, y_train, X_test, y_actual

In [4]:
def RF(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [5]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [6]:
NR_SAMPLES=2000
epsilonl2=['0.1', '0.2', '0.3', '0.4', '0.5']
epsilonl2_labels  = ['0.1', '0.2', '0.3', '0.4', '0.5' ]
epsiloninf=[ '05255', '1255', '2255', '4255' , '8255']
colors=['blue', 'orange', 'green', 'red', 'purple']
length=0
eps=['01', '02', '03', '04', '05']

attacks = ['apgd-ce']

In [7]:
def create_auc(eps, attacks, dataset='cifar10', method='LID', lid_ks=['3', '5', '10', '30', '50']):
    dict_res_rf = {}
    dict_res_lr = {}
    
    if '0.1' in eps:
        norm = 'l2'
    else:
        norm = 'linf'
        
    if dataset == 'cifar10':
        ds = 'cifar10/wrn28-10'
    else:
        ds = 'imagenet/wrn50-2'
    

    table_rf = np.zeros((len(eps), len(lid_ks)))
    table_lr = np.zeros((len(eps), len(lid_ks)))

    for att in attacks:
        auc_list_rf = []
        auc_list_lr = []
        for run_nr in [1,2,3]:
            print("attack", att)
            random_state = np.random.randint(0, 100)
            for i_eps, ep in enumerate(eps):
                for j_eps, lid_k in enumerate(lid_ks):
                    curr_eps = ""
                    if att in ['apgd-ce', 'fgsm', 'bim', 'pgd', 'aa']:
                        curr_eps = "_" + ep
                    
                    clean     = torch.load(os.path.join(base_path.replace('run_1', 'run_{}'.format(run_nr)), ds + '/{}/{}/k{}/{}_normalos_apgd-ce_{}{}.pt'.format(method.lower(), att, lid_k, method.lower(), norm, curr_eps)) ).numpy()[:NR_SAMPLES]
                    chars_adv = torch.load(os.path.join(base_path.replace('run_1', 'run_{}'.format(run_nr)), ds + '/{}/{}/k{}/{}_adverlos_apgd-ce_{}{}.pt'.format(method.lower(), att, lid_k, method.lower(), norm, curr_eps)) ).numpy()[:NR_SAMPLES]

                    characteristics_re     = clean.reshape((clean.shape[0], -1))
                    characteristics_adv_re = chars_adv.reshape((chars_adv.shape[0], -1))

                    X_train, y_train, X_test, y_test = split_data(characteristics_re, characteristics_adv_re, noise=False, test_size=0.2, random_state=random_state)

                    y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)
                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    table_rf[i_eps,j_eps] = auc

                    y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    table_lr[i_eps,j_eps] = auc

            auc_list_rf.append(table_rf.copy())
            auc_list_lr.append(table_lr.copy())

        dict_res_rf[att] = auc_list_rf
        dict_res_lr[att] = auc_list_lr
    
    stacked_rf = np.stack(dict_res_rf[att])
    mean_rf = np.mean(stacked_rf, axis=0)
    var_rf = np.var(stacked_rf, axis=0)
    
    stacked_lr = np.stack(dict_res_lr[att])
    mean_lr = np.mean(stacked_lr, axis=0)
    var_lr = np.var(stacked_lr, axis=0)
    
    return stacked_rf, mean_rf, var_rf, stacked_lr, mean_lr, var_lr

In [23]:
lid_l2_stacked_rf, lid_l2_mean_rf, lid_l2_var_rf, lid_l2_stacked_lr, lid_l2_mean_lr, lid_l2_var_lr = create_auc(eps=epsilonl2, dataset='cifar10', attacks=['apgd-ce'], method='LID', lid_ks=['3', '5', '10', '30', '50'])

attack apgd-ce
attack apgd-ce
attack apgd-ce


In [ ]:
mlid_l2_stacked_rf, mlid_l2_mean_rf, mlid_l2_var_rf, mlid_l2_stacked_lr, mlid_l2_mean_lr, mlid_l2_var_lr = create_auc(eps=epsilonl2, dataset='cifar10', attacks=['apgd-ce'], method='multiLID', lid_ks=['3', '5', '10', '30', '50'])

attack apgd-ce


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

In [ ]:
def plot_fig(stacked_rf, mean_rf, var_rf, stacked_lr, mean_lr, var_lr, epsilons, labels, lid_ks):
    fig, ax = plt.subplots()
    plt.xlabel("$k$")
    plt.ylabel("AUC")
    plt.xticks(lid_ks)
    ks = [3,5,10,30,50]
    ks_arr = np.array(ks)

    plt.title("APGD-CE $L^{2}$ - LID - AUC values per $k$")

    for iter2, ep in enumerate(epsilons):
        plt.fill_between([3,5,10,30,50], mean_rf[:,iter2]-var_rf[:,iter2], mean_rf[:,iter2]+var_rf[:,iter2], alpha=0.2, color=colors[iter2])
        plt.plot(ks_arr, mean_rf[:,iter2], marker='o', markersize=5, alpha=0.5, color=colors[iter2], label=labels[iter2])

    # legend = ax.legend( bbox_to_anchor=(1.0, 0.75), title='RF ($\epsilon$)', loc=6, framealpha=0.3)
    legend = ax.legend( bbox_to_anchor=(1.0, 0.75), title='LID', loc=6, framealpha=0.3)
    plt.gca().add_artist(legend)

    for iter2, ep in enumerate(epsilons):
        plt.fill_between(ks_arr, mean_lr[:,iter2]-var_lr[:,iter2], mean_lr[:,iter2]+var_lr[:,iter2], alpha=0.2, color=colors[iter2] )
        plt.plot(ks_arr, mean_lr[:,iter2], marker='v', markersize=5, alpha=0.5, color=colors[iter2], label=labels[iter2])

    saved = []
    for iter1, ep in enumerate(epsilons):
        saved.append( plt.plot(ks_arr,  np.ones(5)*100, marker='v', alpha=0.5, color=colors[iter1], label=labels[iter1]) )
        
    #legend2 = plt.legend(saved[0] + saved[1]+ saved[2]+ saved[3]+ saved[4], epsilonl2_labels, bbox_to_anchor=(1.0, 0.3),  title='LR ($\epsilon$)',  loc=6, framealpha=0.3)
    legend2 = plt.legend(saved[0]+saved[1]+saved[2]+saved[3]+saved[4], labels, bbox_to_anchor=(1.0, 0.3),  title='multiLID',  loc=6, framealpha=0.3)
    # plt.gca().add_artist(legend2)

    plt.grid(which='major', alpha=0.2)

In [ ]:
plot_fig(
    lid_l2_stacked_lr, 
    lid_l2_mean_lr, 
    lid_l2_var_lr, 
    mlid_l2_stacked_rf, 
    mlid_l2_mean_rf, 
    mlid_l2_var_rf, 
    epsilonl2, epsilonl2_labels, [3, 5, 10, 30, 50])
# plt.savefig("plots/lines_over_k/fill_apgd-ce_l2.png", bbox_inches='tight', dpi=300)

In [19]:
mLid_linf_stacked_rf, mLid_linf_mean_rf, mLid_var_rf, mLid_linf_stacked_linf_lr, mLid_linf_mean_lr, mLid_linf_var_lr = create_auc(eps=epsiloninf, attacks=['apgd-ce'], dataset='cifar10', method='multiLID', lid_ks=['3', '5', '10', '30', '50'])

attack apgd-ce


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

attack apgd-ce


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

attack apgd-ce


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/